# Visualize the result of a list of GANs

In [ ]:
# general setup
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
# for compatible with python 3
from __future__ import print_function
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Deep Convolutional GAN

In [ ]:
from dcgan import DCGAN
from utils import load_train_data, load_test_data

In [ ]:
# load cifar10 dataset
train_samples = load_train_data() / 255.0

In [ ]:
dcgan = DCGAN()
if False:
    dcgan.discriminator.summary()
    dcgan.generator.summary()
    dcgan.discriminator_generator.summary()

In [ ]:
dis_losses, gen_losses = dcgan.train(train_samples)

In [ ]:
from utils import viz_grid
# visualize
tracked_noise = np.random.normal(0, 1, [64, 64])
fig = plt.figure(figsize = (8, 8))   
ax1 = plt.subplot(111)
ax1.imshow(viz_grid(dcgan.generate(tracked_noise), 1))
plt.show()

plt.plot(dis_losses)
plt.title('discriminator loss')
plt.xlabel('iterations')
plt.ylabel('loss')
plt.show()

plt.plot(gen_losses)
plt.title('generator loss')
plt.xlabel('iterations')
plt.ylabel('loss')
plt.show()

## Conditional GAN (condition on the class labels)

In [ ]:
from cgan import CGAN
from utils import get_CIFAR10_data, to_categorical

In [ ]:
# load cifar10 data with labels
cifar10 = get_CIFAR10_data(subtract_mean=False, channels_first=False)
X_train = cifar10['X_train'] / 255.0
y_train = cifar10['y_train']
Y_train = to_categorical(y_train)

In [ ]:
cgan = CGAN()
if False:
    cgan.discriminator.summary()
    cgan.generator.summary()
    cgan.discriminator_generator.summary()

In [ ]:
dis_losses, gen_losses = cgan.train(X_train, Y_train)